In [39]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from IPython.display import Image

import glob
import re
import os
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import statistics
import collections
import itertools
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import gc
import time
import math
from scipy import signal
from scipy.fftpack import fft, fftfreq
from scipy.stats import iqr
from scipy.stats import pearsonr
from scipy.stats import skew
from scipy.stats import kurtosis
from spectrum import *
from statsmodels.robust import mad as median_deviation
from sklearn.preprocessing import minmax_scale
import os

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.manifold import TSNE
from lightgbm import LGBMClassifier
from collections import Counter

In [64]:
# 混同行列
def generate_cm(df, index, columns, filename, index_name="True Label", columns_name="Estimated Label", figure_size=(10, 8), fontsize=20,  font="Times New Roman", bottom=None, top=None, right=None, left=None):
    cm2 = pd.DataFrame(df, index=index, columns=columns)
    cm3 = cm2.astype('float').div(cm2.sum(axis=1), axis="index")
    cm3_object = pd.DataFrame(cm3.values, index=index, columns=columns, dtype=str)
    
    for i in range(cm3_object.shape[0]):
        for j in range(cm3_object.shape[1]):
            cm3_object.iat[i, j] = "{0:.1f}".format(cm3.iat[i, j]) + "\n" + "(" + str(cm2.iat[i, j]) + ")"
    
    cm3.index.name = "True label"
    cm3.columns.name = "Estimated label"
    sns.set()
    sns.set('talk', 'whitegrid', 'dark', font_scale=1)

    plt.figure()
    plt.rcParams["font.size"] = fontsize
    plt.rcParams['font.family'] = font
    plt.figure(figsize=figure_size)
    plt.xlabel('Predict')
    plt.ylabel('Actual')

#     plt.subplots_adjust(bottom=bottom, top=top, left=left, right=right)
    plt.tight_layout()
    #     plt.subplots_adjust()

    sns.heatmap(cm3, annot=cm3_object.values, fmt="", linewidth=2, cmap="gray_r", linecolor="black", vmax=1, vmin=0)


    plt.savefig(filename, bbox_inches="tight", pad_inches=0.05)
    plt.close("all")

In [14]:
def run_closs_val(terget_df, terget_dir, terget_class, condition, classifiers, features_list, val_type):
    
    
    X = terget_df[ features_list].values
    Y = terget_df.label.values
    #groups = terget_df.case.values

    labels = np.unique(Y)
    
    for i in range(len(classifiers)):
        name, clf = classifiers[i]
        print(name)
        scaler = StandardScaler()
        #pca = PCA(n_components=0.95)
        pipeline = make_pipeline(scaler, clf)
        #cv = LeaveOneGroupOut()
        #cv = StratifiedKFold(10)
        if (val_type == "person-out"):
            print(val_type)
            cv = LeaveOneGroupOut()
        else:
            print("10 fold")
            cv = StratifiedKFold(10)


        
        y_pred = cross_val_predict(pipeline, X, Y, #groups=groups, 
                                   cv=cv, n_jobs=7, verbose=1)

        cr_dict = classification_report(Y, y_pred, output_dict=True)
        cr_df = pd.DataFrame(cr_dict).T
        cr_df.to_csv(terget_dir + "/"+condition+ '/clf_reports' + '/'+condition + '_'+ terget_class + '_'+'result_report_'+name+'.csv')

        cm = confusion_matrix(Y, y_pred, labels=labels)
        cm_df = pd.DataFrame(cm, index=labels, columns=labels)
        cm_file_name =  terget_dir+ "/"+condition+ '/'+condition + '_'+ terget_class + '_'+ 'result_cm_' + name + ".png"

        generate_cm(cm_df, index=labels, columns=labels, filename = cm_file_name)
        

#         ddf = pd.concat([pd.Series(y_pred), pd.Series(Y), pd.Series(groups)] ,axis = 1)
#         ddf.columns=["y_pred","y", "session"]
        
        
        ddf = pd.concat([pd.Series(y_pred), pd.Series(Y)] ,axis = 1)
        ddf.columns=["y_pred","y"]
        
        ddf.to_csv(terget_dir+ "/"+condition+ '/'+condition + '_'+ terget_class + '_'+ 'y_pred_true_user_' + name +'.csv')
        
        print(name,"fin")
    #     Image(cm_file_name)

In [50]:
def run_closs_val_PI(terget_df, terget_dir, terget_class, condition, classifiers, features_list):
    
    
    X = terget_df[ features_list].values
    Y = terget_df.label.values
        
    groups = terget_df.user.values
    labels = np.unique(Y)
    #val_type = condition.split("_")[-1]
    
    
    for i in range(len(classifiers)):
        name, clf = classifiers[i]
        print(name)
        scaler = StandardScaler()
        #pca = PCA(n_components=0.95)
        pipeline = make_pipeline(scaler, clf)
        
        
        cv = LeaveOneGroupOut()
#         if (val_type == "person-out"):
#             print(val_type)
#             cv = LeaveOneGroupOut()
#         else:
#             print("10 fold")
#             cv = StratifiedKFold(10)

        # cv = StratifiedKFold(5)
        y_pred = cross_val_predict(pipeline, X, Y,
                                   groups=groups, cv=cv,
                                   n_jobs=7, verbose=1)

        cr_dict = classification_report(Y, y_pred, output_dict=True)
        cr_df = pd.DataFrame(cr_dict).T
        cr_df.to_csv(terget_dir + "/"+condition+ '/clf_reports' + '/'+condition + '_'+ terget_class + '_'+'result_report_'+name+'.csv')

        cm = confusion_matrix(Y, y_pred, labels=labels)
        cm_df = pd.DataFrame(cm, index=labels, columns=labels)
        cm_file_name =  terget_dir+ "/"+condition+ '/'+condition + '_'+ terget_class + '_'+ 'result_cm_' + name + ".png"

        generate_cm(cm_df, index=labels, columns=labels, filename = cm_file_name)
        

        ddf = pd.concat([pd.Series(y_pred), pd.Series(Y), pd.Series(groups)] ,axis = 1)
        ddf.columns=["y_pred","y","user"]
        ddf.to_csv(terget_dir+ "/"+condition+ '/'+condition + '_'+ terget_class + '_'+ 'y_pred_true_user_' + name +'.csv')
        
        print(name,"fin")
    #     Image(cm_file_name)

In [3]:
all_feature145_names = ['t-Mean-1',
 't-Mean-2',
 't-Mean-3',
 't-STD-1',
 't-STD-2',
 't-STD-3',
 't-Mad-1',
 't-Mad-2',
 't-Mad-3',
 't-Max-1',
 't-Max-2',
 't-Max-3',
 't-Min-1',
 't-Min-2',
 't-Min-3',
 't-SMA-1',
 't-Energy-1',
 't-Energy-2',
 't-Energy-3',
 't-IQR-1',
 't-IQR-2',
 't-IQR-3',
 't-ropy-1',
 't-ropy-2',
 't-ropy-3',
 't-ARCoeff-1',
 't-ARCoeff-2',
 't-ARCoeff-3',
 't-ARCoeff-4',
 't-ARCoeff-5',
 't-ARCoeff-6',
 't-ARCoeff-7',
 't-ARCoeff-8',
 't-ARCoeff-9',
 't-ARCoeff-10',
 't-ARCoeff-11',
 't-ARCoeff-12',
 't-Correlation-1',
 't-Correlation-2',
 't-Correlation-3',
 't-Mean-mag',
 't-STD-mag',
 't-Mad-mag',
 't-Max-mag',
 't-Min-mag',
 't-SMA-mag',
 't-Energy-mag',
 't-IQR-mag',
 't-ropy-mag',
 't-ARCoeff-mag1',
 't-ARCoeff-mag2',
 't-ARCoeff-mag3',
 't-ARCoeff-mag4',
 'f-Mean-1',
 'f-Mean-2',
 'f-Mean-3',
 'f-STD-1',
 'f-STD-2',
 'f-STD-3',
 'f-Mad-1',
 'f-Mad-2',
 'f-Mad-3',
 'f-Max-1',
 'f-Max-2',
 'f-Max-3',
 'f-Min-1',
 'f-Min-2',
 'f-Min-3',
 'f-SMA-1',
 'f-Energy-1',
 'f-Energy-2',
 'f-Energy-3',
 'f-IQR-1',
 'f-IQR-2',
 'f-IQR-3',
 'f-ropy-1',
 'f-ropy-2',
 'f-ropy-3',
 'f-MaxInds-1',
 'f-MaxInds-2',
 'f-MaxInds-3',
 'f-MeanFreq-1',
 'f-MeanFreq-2',
 'f-MeanFreq-3',
 'f-Skewness-1',
 'f-Skewness-2',
 'f-Skewness-3',
 'f-Kurtosis-1',
 'f-Kurtosis-2',
 'f-Kurtosis-3',
 'f-BandsEnergyOld-1',
 'f-BandsEnergyOld-2',
 'f-BandsEnergyOld-3',
 'f-BandsEnergyOld-4',
 'f-BandsEnergyOld-5',
 'f-BandsEnergyOld-6',
 'f-BandsEnergyOld-7',
 'f-BandsEnergyOld-8',
 'f-BandsEnergyOld-9',
 'f-BandsEnergyOld-10',
 'f-BandsEnergyOld-11',
 'f-BandsEnergyOld-12',
 'f-BandsEnergyOld-13',
 'f-BandsEnergyOld-14',
 'f-BandsEnergyOld-15',
 'f-BandsEnergyOld-16',
 'f-BandsEnergyOld-17',
 'f-BandsEnergyOld-18',
 'f-BandsEnergyOld-19',
 'f-BandsEnergyOld-20',
 'f-BandsEnergyOld-21',
 'f-BandsEnergyOld-22',
 'f-BandsEnergyOld-23',
 'f-BandsEnergyOld-24',
 'f-BandsEnergyOld-25',
 'f-BandsEnergyOld-26',
 'f-BandsEnergyOld-27',
 'f-BandsEnergyOld-28',
 'f-BandsEnergyOld-29',
 'f-BandsEnergyOld-30',
 'f-BandsEnergyOld-31',
 'f-BandsEnergyOld-32',
 'f-BandsEnergyOld-33',
 'f-BandsEnergyOld-34',
 'f-BandsEnergyOld-35',
 'f-BandsEnergyOld-36',
 'f-BandsEnergyOld-37',
 'f-BandsEnergyOld-38',
 'f-BandsEnergyOld-39',
 'f-BandsEnergyOld-40',
 'f-BandsEnergyOld-41',
 'f-BandsEnergyOld-42',
 'f-Mean-mag',
 'f-STD-mag',
 'f-Mad-mag',
 'f-Max-mag',
 'f-Min-mag',
 'f-SMA-mag',
 'f-Energy-mag',
 'f-IQR-mag',
 'f-ropy-mag',
 'f-MaxInds-mag',
 'f-MeanFreq-mag',
 'f-Skewness-mag',
 'f-Kurtosis-mag']



def add_acc(n):
    return "acc_" + n

acc_feature145_names = list(map(add_acc, all_feature145_names))

In [4]:
classifiers = [
               ("RF", RandomForestClassifier(random_state=0)),
               ("ANN", MLPClassifier()),
               ("LR", LogisticRegression()),
               ("DT", DecisionTreeClassifier()),
               ("ET", ExtraTreeClassifier()),
               ("LR", LogisticRegression()),
               ("NB", GaussianNB()),
               ("SVM", SVC()),
               ("KNN", KNeighborsClassifier()),
               ("LightGBM", LGBMClassifier())]

/Users/y-nakamura/yugo_workspace/bar_exercise/data/raw/20210716/Bottom_2021-07-16T15.18.45.753_E2EE3F7D14CE_Accelerometer.csv
/Users/y-nakamura/yugo_workspace/bar_exercise/data/raw/20210716/Middle_2021-07-16T15.18.10.660_CEF5C0D2C881_Accelerometer.csv
/Users/y-nakamura/yugo_workspace/bar_exercise/data/raw/20210716/Upper_2021-07-16T15.17.48.292_D738727472C3_Accelerometer.csv

In [21]:
os.getcwd()

terger_dir ="/Users/y-nakamura/yugo_workspace/bar_exercise/data/features/20210716"

os.chdir(terger_dir)

# df_bottom = pd.read_csv(terger_dir + "/Bottom_2021-07-16T15.18.45.753_E2EE3F7D14CE_Accelerometer_features.csv")
# df_middle = pd.read_csv(terger_dir + "/Middle_2021-07-16T15.18.10.660_CEF5C0D2C881_Accelerometer_features.csv")
# df_upper = pd.read_csv(terger_dir + "/Upper_2021-07-16T15.17.48.292_D738727472C3_Accelerometer_features.csv")
df_upper = pd.read_csv(terger_dir + "/0721_Upper_2021-07-16T15.17.48.292_D738727472C3_Accelerometer - Upper_2021-07-16T15.17.48.292_D738727472C3_Accelerometer_features.csv")


df_upper.head()

,acc_t-Mean-1,acc_t-Mean-2,acc_t-Mean-3,acc_t-STD-1,acc_t-STD-2,acc_t-STD-3,acc_t-Mad-1,acc_t-Mad-2,acc_t-Mad-3,acc_t-Max-1,...,acc_f-SMA-mag,acc_f-Energy-mag,acc_f-IQR-mag,acc_f-ropy-mag,acc_f-MaxInds-mag,acc_f-MeanFreq-mag,acc_f-Skewness-mag,acc_f-Kurtosis-mag,label,user
0,0.012277,-0.980031,-0.221554,0.000512,0.001252,0.000860,0.000000,0.000000,0.000000,0.013,...,0.063058,0.126212,0.000341,-1.929943,0.0,0.061105,5.388156,27.032238,0,0
1,0.012477,-0.980200,-0.221615,0.000529,0.000532,0.000798,0.001483,0.000000,0.001483,0.013,...,0.062917,0.126258,0.000098,-1.982123,0.0,0.028830,5.388158,27.032254,0,0
2,0.012338,-0.980077,-0.221523,0.000686,0.000771,0.000861,0.001483,0.001483,0.001483,0.013,...,0.062943,0.126222,0.000113,-1.969639,0.0,0.034847,5.388158,27.032249,0,0
3,0.012262,-0.979815,-0.221246,0.000663,0.000839,0.001023,0.001483,0.000000,0.001483,0.013,...,0.062945,0.126143,0.000146,-1.960250,0.0,0.043273,5.388158,27.032248,0,0
4,0.012262,-0.980000,-0.221262,0.000589,0.000823,0.000933,0.000000,0.000000,0.001483,0.013,...,0.062957,0.126189,0.000130,-1.960811,0.0,0.044567,5.388158,27.032248,0,0


In [26]:
np.unique(df_upper.user.values)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [29]:
user_list = [1, 2, 3, 4, 5, 6, 7, 8]

In [35]:
df = df_upper[df_upper.user.isin(user_list)]

In [37]:
df 

,acc_t-Mean-1,acc_t-Mean-2,acc_t-Mean-3,acc_t-STD-1,acc_t-STD-2,acc_t-STD-3,acc_t-Mad-1,acc_t-Mad-2,acc_t-Mad-3,acc_t-Max-1,...,acc_f-SMA-mag,acc_f-Energy-mag,acc_f-IQR-mag,acc_f-ropy-mag,acc_f-MaxInds-mag,acc_f-MeanFreq-mag,acc_f-Skewness-mag,acc_f-Kurtosis-mag,label,user
122,0.011815,-0.980262,-0.221969,0.000893,0.000790,0.002267,0.000000,0.001483,0.002965,0.014,...,0.062956,0.126291,0.000091,-1.971255,0.0,0.046497,5.388159,27.032254,0,1
123,0.011954,-0.980415,-0.222108,0.000643,0.000654,0.002444,0.000000,0.001483,0.002965,0.013,...,0.062968,0.126337,0.000099,-1.971716,0.0,0.045397,5.388159,27.032256,0,1
124,0.012031,-0.980062,-0.222523,0.000723,0.000742,0.002348,0.001483,0.001483,0.002965,0.013,...,0.062976,0.126274,0.000138,-1.962763,0.0,0.046902,5.388158,27.032251,0,1
125,0.011738,-0.980077,-0.222108,0.000770,0.000917,0.002463,0.001483,0.001483,0.002965,0.013,...,0.062970,0.126253,0.000185,-1.963121,0.0,0.045907,5.388158,27.032250,0,1
126,0.011815,-0.980108,-0.222585,0.000821,0.001054,0.004235,0.000000,0.001483,0.004448,0.014,...,0.062943,0.126289,0.000080,-1.976088,0.0,0.035287,5.388158,27.032253,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2075,0.010246,-0.978938,-0.225092,0.013863,0.002882,0.012576,0.010378,0.004448,0.016309,0.034,...,0.062998,0.126181,0.000198,-1.945451,0.0,0.069499,5.388158,27.032250,0,2
2076,0.010292,-0.978969,-0.225062,0.014791,0.002762,0.012091,0.017791,0.004448,0.016309,0.034,...,0.062995,0.126189,0.000131,-1.946485,0.0,0.068486,5.388158,27.032254,0,2
2077,0.010169,-0.978477,-0.226108,0.009283,0.002106,0.009401,0.011861,0.002965,0.014826,0.025,...,0.062956,0.126103,0.000146,-1.951031,0.0,0.065074,5.388158,27.032254,0,2
2078,0.009938,-0.978677,-0.224800,0.006079,0.001599,0.007685,0.008896,0.001483,0.008896,0.020,...,0.062951,0.126068,0.000193,-1.949611,0.0,0.055145,5.388158,27.032251,0,2


In [40]:
Counter(list(df.label))

Counter({0: 424, 1: 163, 2: 250, 3: 265, 4: 204})

In [7]:
#np.unique(df_bottom.label)

In [8]:
#np.unique(df_middle.label)

In [9]:
#np.unique(df_upper.label)

In [41]:
target_dir = "/Users/y-nakamura/yugo_workspace/bar_exercise/data/clf_results/20210716"
os.chdir(target_dir)

In [42]:
val_type =  "person-out"
#val_type =  "10 fold"

In [48]:
condition = "upper_PI_window64slide32_10-fold"
target_class = "bar_exercise"
os.makedirs(condition, exist_ok=True)
os.makedirs(condition +'/clf_reports', exist_ok=True)

In [53]:
features_list = acc_feature145_names
run_closs_val_PI(df, target_dir, target_class, condition, classifiers, features_list)

RF


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.6s remaining:    0.6s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    1.0s finished


RF fin
ANN


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    1.1s remaining:    1.1s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    1.8s finished


ANN fin
LR


...

SVM fin
KNN


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    0.0s finished


KNN fin
LightGBM


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    3.6s remaining:    3.6s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    5.5s finished


LightGBM fin


In [26]:
condition = "middle_window64slide32_10-fold"
target_class = "bar_exercise"
os.makedirs(condition, exist_ok=True)
os.makedirs(condition +'/clf_reports', exist_ok=True)

features_list = acc_feature145_names
run_closs_val(df_middle, target_dir, target_class, condition, classifiers, features_list, val_type)

RF
10 fold


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    2.1s remaining:    0.5s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    2.1s finished


RF fin
ANN
10 fold


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    4.5s remaining:    1.1s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    4.5s finished


ANN fin
LR
10 fold


...

SVM fin
KNN
10 fold


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    0.1s finished


KNN fin
LightGBM
10 fold


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    5.3s remaining:    1.3s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    5.3s finished


LightGBM fin


In [27]:
condition = "upper_window64slide32_10-fold"
target_class = "bar_exercise"
os.makedirs(condition, exist_ok=True)
os.makedirs(condition +'/clf_reports', exist_ok=True)

features_list = acc_feature145_names
run_closs_val(df_upper, target_dir, target_class, condition, classifiers, features_list, val_type)

RF
10 fold


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    1.6s remaining:    0.4s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    1.6s finished


RF fin
ANN
10 fold


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    3.0s remaining:    0.8s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    3.0s finished


ANN fin
LR
10 fold


...

SVM fin
KNN
10 fold


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    0.1s finished


KNN fin
LightGBM
10 fold


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    5.2s remaining:    1.3s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    5.4s finished


LightGBM fin


# Majority vote

In [54]:
def sliding_window(data, window, step_size):
    shape = (int(data.shape[-1] / window * window / step_size - 1), window)
    strides = (data.strides[-1] * step_size, data.strides[-1])
    return np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)

def majority_filter(y_pred, window_size):
    sliding_y_pred  = sliding_window(y_pred, window_size, 1)
    voted_y_pred = []
    for y_list in sliding_y_pred:
        voted_y_pred.append(collections.Counter(y_list).most_common()[0][0])
    voted_y_pred.append(y_list[-1])
    
    return voted_y_pred

In [56]:
def run_majority_vote(target_dir, target_class, in_condition, out_condition, vote_num):
    print(in_condition, ",", out_condition)
    os.makedirs(out_condition, exist_ok=True)
    target_files = [tf for tf in os.listdir(target_dir + "/" + in_condition )  if tf.startswith( in_condition+ '_'+target_class +'_y_pred')]
    target_files.sort()
    print(target_files)
    print(target_files[0])
    os.makedirs(out_condition +'/clf_reports', exist_ok=True)
    for target_file in target_files:
        df = pd.read_csv(target_dir  + "/" + in_condition + "/"+ target_file, index_col=[0])
        model_name = target_file[:-4].split("_")[-1]
        #print(model_name)

        output_file = target_dir + "/"+out_condition + '/'+out_condition + '_'+ target_class + '_y_pred_true_user_' + model_name +'.csv'

            
        df.y_pred = majority_filter(df.y_pred.values, vote_num)

        df.to_csv(output_file)

        labels = np.unique(df.y)

        cr_dict = classification_report(df.y, df.y_pred, output_dict=True)

        cr_df = pd.DataFrame(cr_dict).T

        cr_df.to_csv( target_dir + "/"+out_condition+ '/clf_reports' + '/'+out_condition + '_'+ target_class+ '_result_report_'+model_name+'.csv')

        cm = confusion_matrix(df.y, df.y_pred, labels=labels)
        cm_df = pd.DataFrame(cm, index=labels, columns=labels)
        cm_file_name =  target_dir + "/"+out_condition+ '/'+out_condition + '_'+ target_class + '_result_cm_' + model_name + ".png"
        generate_cm(cm_df, index=labels, columns=labels, filename = cm_file_name)


In [57]:
condition = "upper_window64slide32_LOPO"
target_class = "bar_exercise"
os.makedirs(condition, exist_ok=True)
os.makedirs(condition +'/clf_reports', exist_ok=True)

features_list = acc_feature145_names
run_closs_val_PI(df, target_dir, target_class, condition, classifiers, features_list)

RF


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.7s remaining:    0.7s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    1.1s finished


RF fin
ANN


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    1.1s remaining:    1.1s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    1.7s finished


ANN fin
LR


...

SVM fin
KNN


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    0.0s finished


KNN fin
LightGBM


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    3.4s remaining:    3.4s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    5.3s finished


LightGBM fin


In [65]:
in_condition = "upper_window64slide32_LOPO"
vote_num = 10
out_condition = "upper_window64slide32_LOPO_majority" + str(vote_num)

target_class = "bar_exercise"
run_majority_vote(target_dir, target_class, in_condition, out_condition, vote_num)


upper_window64slide32_LOPO , upper_window64slide32_LOPO_majority10
['upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_ANN.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_DT.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_ET.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_KNN.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_LR.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_LightGBM.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_NB.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_RF.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_SVM.csv']
upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_ANN.csv


In [66]:
in_condition = "upper_window64slide32_LOPO"
vote_num = 5
out_condition = "upper_window64slide32_LOPO_majority" + str(vote_num)

target_class = "bar_exercise"
run_majority_vote(target_dir, target_class, in_condition, out_condition, vote_num)


upper_window64slide32_LOPO , upper_window64slide32_LOPO_majority5
['upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_ANN.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_DT.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_ET.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_KNN.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_LR.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_LightGBM.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_NB.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_RF.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_SVM.csv']
upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_ANN.csv


In [ ]:
in_condition = "upper_window64slide32_LOPO"
vote_num = 3
out_condition = "upper_window64slide32_LOPO_majority" + str(vote_num)

target_class = "bar_exercise"
run_majority_vote(target_dir, target_class, in_condition, out_condition, vote_num)
